In [4]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast

In [20]:
# size_width_rugs_runners
# size_length_rugs_runners
# size_diameter_rugs_runners

# parameters
customer_id = '5'
formatted_attribute = 'certifications'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Pendant Lighting"""


# regex_pattern = r"(?i)(\s[^t]\d+\-Inch x \d+\-Inch)|()"
regex_pattern = r"(?i)(\s[^t]\d+\-Inch x \d+\-Inch)|(\d+\\\" x \d*\\\")"


value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [21]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [22]:
df = query_from_file(file_name='query/bbby.sql', params=params)
df

attribute                                       value  \
0    certifications  ["AHAM Certified","Energy Star Certified"]   
1    certifications                       CADR Certified Rating   
2    certifications                       CADR Certified Rating   
3    certifications                       CADR Certified Rating   
4    certifications             ["CARB Certified","ETL Listed"]   
..              ...                                         ...   
107  certifications                                         n/a   
108  certifications                                         n/a   
109  certifications                                         n/a   
110  certifications                                         n/a   
111  certifications                                         n/a   

       customers_name external_id  \
0    bedbathandbeyond    16946680   
1    bedbathandbeyond    62141956   
2    bedbathandbeyond    68136994   
3    bedbathandbeyond    69597097   
4    bedbathandbeyond    69597306   
..                ...         ...   
107  bedbathandbeyond    69662296   
108  bedbathandbeyond    69662297   
109  bedbathandbeyond    69662298   
110  bedbathandbeyond    69662299   
111  bedbathandbeyond    69662300   

                                                                                   long_desc  \
0                             GermGuardian® 28-Inch 3-in-1 HEPA Tower with UV-C Air Purifier   
1    GermGuardian® AC5350W Digital Air Purifier 28-Inch Tower with True HEPA Filter and UV-C   
2                                          Dyson® Pure Cool™ DP04 Air Purifier Fan in Silver   
3                                                     PureZone 3-in-1 True HEPA Air Purifier   
4                                       Brondell O2+ Horizon True HEPA Air Purifier in White   
..                                                                                       ...   
107                            Essick Air AIRCARE Aria 0.75 Gallon Steam Humidifier in White   
108                  Essick Air AIRCARE ESSENTIALS DUET Ultrasonic Humidifier in White/Green   
109                                   Essick Air AIRCARE Nova Ultrasonic Humidifier in Black   
110                      Essick Air AIRCARE ESSENTIALS Ozark Steam Humidifier in White/Green   
111                                 Essick Air AIRCARE Pillar Ultrasonic Humidifier in Beige   

    resolution  
0     standard  
1     standard  
2     standard  
3     standard  
4     standard  
..         ...  
107   standard  
108   standard  
109   standard  
110   standard  
111   standard  

[112 rows x 6 columns]

In [23]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['AHAM Certified',
 'CADR Certified Rating',
 'CARB Certified',
 'CE Marketing',
 'CertiPUR-US Certified Foam',
 'DLC Approved',
 'EnergyStar',
 'Energy Star Certified',
 'ETL Certification',
 'ETL Listed',
 'FCC Certification',
 'Meets CPSC Standards',
 'RoHS Certification',
 'UL Listed']

In [24]:
metric=""
range_string =""""

 Greater than 108
 12 - 23.9
 24 - 35.9
 36 - 47.9
 48 - 59.9
 60 - 71.9
 72 - 83.9
 84 - 95.9
 96 - 107.9
 0 - 11.9
 """

In [26]:
kwp=KeywordProcessor()
print(kwp.add_keywords_from_list(list_values))
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

None


0      []
1      []
2      []
3      []
4      []
       ..
107    []
108    []
109    []
110    []
111    []
Name: long_desc, Length: 112, dtype: object

In [27]:
df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
df['matches'].explode().value_counts()

Series([], Name: matches, dtype: int64)

In [28]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [29]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df

attribute                                       value  \
0    certifications  ["AHAM Certified","Energy Star Certified"]   
1    certifications                       CADR Certified Rating   
2    certifications                       CADR Certified Rating   
3    certifications                       CADR Certified Rating   
4    certifications             ["CARB Certified","ETL Listed"]   
..              ...                                         ...   
107  certifications                                         n/a   
108  certifications                                         n/a   
109  certifications                                         n/a   
110  certifications                                         n/a   
111  certifications                                         n/a   

       customers_name external_id  \
0    bedbathandbeyond    16946680   
1    bedbathandbeyond    62141956   
2    bedbathandbeyond    68136994   
3    bedbathandbeyond    69597097   
4    bedbathandbeyond    69597306   
..                ...         ...   
107  bedbathandbeyond    69662296   
108  bedbathandbeyond    69662297   
109  bedbathandbeyond    69662298   
110  bedbathandbeyond    69662299   
111  bedbathandbeyond    69662300   

                                                                                   long_desc  \
0                             GermGuardian® 28-Inch 3-in-1 HEPA Tower with UV-C Air Purifier   
1    GermGuardian® AC5350W Digital Air Purifier 28-Inch Tower with True HEPA Filter and UV-C   
2                                          Dyson® Pure Cool™ DP04 Air Purifier Fan in Silver   
3                                                     PureZone 3-in-1 True HEPA Air Purifier   
4                                       Brondell O2+ Horizon True HEPA Air Purifier in White   
..                                                                                       ...   
107                            Essick Air AIRCARE Aria 0.75 Gallon Steam Humidifier in White   
108                  Essick Air AIRCARE ESSENTIALS DUET Ultrasonic Humidifier in White/Green   
109                                   Essick Air AIRCARE Nova Ultrasonic Humidifier in Black   
110                      Essick Air AIRCARE ESSENTIALS Ozark Steam Humidifier in White/Green   
111                                 Essick Air AIRCARE Pillar Ultrasonic Humidifier in Beige   

    resolution matches  
0     standard      []  
1     standard      []  
2     standard      []  
3     standard      []  
4     standard      []  
..         ...     ...  
107   standard      []  
108   standard      []  
109   standard      []  
110   standard      []  
111   standard      []  

[112 rows x 7 columns]

In [30]:
df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
df

attribute                                       value  \
0    certifications  ["AHAM Certified","Energy Star Certified"]   
1    certifications                       CADR Certified Rating   
2    certifications                       CADR Certified Rating   
3    certifications                       CADR Certified Rating   
4    certifications             ["CARB Certified","ETL Listed"]   
..              ...                                         ...   
107  certifications                                         n/a   
108  certifications                                         n/a   
109  certifications                                         n/a   
110  certifications                                         n/a   
111  certifications                                         n/a   

       customers_name external_id  \
0    bedbathandbeyond    16946680   
1    bedbathandbeyond    62141956   
2    bedbathandbeyond    68136994   
3    bedbathandbeyond    69597097   
4    bedbathandbeyond    69597306   
..                ...         ...   
107  bedbathandbeyond    69662296   
108  bedbathandbeyond    69662297   
109  bedbathandbeyond    69662298   
110  bedbathandbeyond    69662299   
111  bedbathandbeyond    69662300   

                                                                                   long_desc  \
0                             GermGuardian® 28-Inch 3-in-1 HEPA Tower with UV-C Air Purifier   
1    GermGuardian® AC5350W Digital Air Purifier 28-Inch Tower with True HEPA Filter and UV-C   
2                                          Dyson® Pure Cool™ DP04 Air Purifier Fan in Silver   
3                                                     PureZone 3-in-1 True HEPA Air Purifier   
4                                       Brondell O2+ Horizon True HEPA Air Purifier in White   
..                                                                                       ...   
107                            Essick Air AIRCARE Aria 0.75 Gallon Steam Humidifier in White   
108                  Essick Air AIRCARE ESSENTIALS DUET Ultrasonic Humidifier in White/Green   
109                                   Essick Air AIRCARE Nova Ultrasonic Humidifier in Black   
110                      Essick Air AIRCARE ESSENTIALS Ozark Steam Humidifier in White/Green   
111                                 Essick Air AIRCARE Pillar Ultrasonic Humidifier in Beige   

    resolution matches  
0     standard      []  
1     standard      []  
2     standard      []  
3     standard      []  
4     standard      []  
..         ...     ...  
107   standard      []  
108   standard      []  
109   standard      []  
110   standard      []  
111   standard      []  

[112 rows x 7 columns]

In [31]:
x=df[df['matches'].astype(str)!='[]']
x

Empty DataFrame
Columns: [attribute, value, customers_name, external_id, long_desc, resolution, matches]
Index: []

In [12]:
x['matches']=x['matches'].str.get(0)
x['matches']=x['matches'].apply(lambda x:x.lower())
x[['length', 'width']]=x['matches'].str.split(" x ",expand=True,)
x

<ipython-input-12-91d081f8f086>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['matches']=x['matches'].str.get(0)
<ipython-input-12-91d081f8f086>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['matches']=x['matches'].apply(lambda x:x.lower())
C:\Users\groupby\anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

attribute       value    customers_name external_id  \
52     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    61327016   
68     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    63730005   
69     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    63730043   
70     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    63730081   
71     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    63730142   
...                         ...         ...               ...         ...   
57976  size_length_rugs_runners         n/a  bedbathandbeyond    69649155   
57977  size_length_rugs_runners         n/a  bedbathandbeyond    69649156   
57978  size_length_rugs_runners         n/a  bedbathandbeyond    69649157   
57979  size_length_rugs_runners         n/a  bedbathandbeyond    69672237   
57980  size_length_rugs_runners         n/a  bedbathandbeyond    69672241   

                                                             long_desc  \
52                Capel Rugs® Gava Stripes 5\" x 8\" Area Rug in Beige   
68              Weather Guard™ Squares 34.5\" x 59\" Door Mat in Camel   
69         Weather Guard™ Squares 34.5\" x 59\" Door Mat in Dark Brown   
70           Weather Guard™ Squares 34.5\" x 59\" Door Mat in Charcoal   
71        Weather Guard™ Squares 34.5\" x 59\" Door Mat in Medium Grey   
...                                                                ...   
57976  The Farm Rooster 18\" x 30\" Anti-Fatigue Kitchen Mat in Walnut   
57977  Jolly Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Rain Drum   
57978  The Farm Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   
57979                    Wamsutta® Preston 24\" x 40\" Bath Rug in Sea   
57980                         Preston 17\" x 24\" Bath Rug in New Blue   

      resolution      matches length width  
52      standard    5\" x 8\"    5\"   8\"  
68      standard   5\" x 59\"    5\"  59\"  
69      standard   5\" x 59\"    5\"  59\"  
70      standard   5\" x 59\"    5\"  59\"  
71      standard   5\" x 59\"    5\"  59\"  
...          ...          ...    ...   ...  
57976   standard  18\" x 30\"   18\"  30\"  
57977   standard  22\" x 35\"   22\"  35\"  
57978   standard  22\" x 35\"   22\"  35\"  
57979   standard  24\" x 40\"   24\"  40\"  
57980   standard  17\" x 24\"   17\"  24\"  

[7343 rows x 9 columns]

In [13]:
x['length']=x['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-\"','',str(measure)))
x['width']=x['width'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-\"','',str(measure)))
x['length']=x['length'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
x['width']=x['width'].apply(lambda quote: quote.replace('"', ''))
x['length']=x['length'].replace('"', '').apply(lambda quote: quote.replace('-',''))
x['width']=x['width'].apply(lambda quote: quote.replace('-',''))
x['length']=x['length'].astype(int)
x['width']=x['width'].astype(int)
x

<ipython-input-13-31601631676c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['length']=x['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-\"','',str(measure)))
<ipython-input-13-31601631676c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['width']=x['width'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-\"','',str(measure)))
<ipython-input-13-31601631676c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

attribute       value    customers_name external_id  \
52     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    61327016   
68     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    63730005   
69     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    63730043   
70     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    63730081   
71     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    63730142   
...                         ...         ...               ...         ...   
57976  size_length_rugs_runners         n/a  bedbathandbeyond    69649155   
57977  size_length_rugs_runners         n/a  bedbathandbeyond    69649156   
57978  size_length_rugs_runners         n/a  bedbathandbeyond    69649157   
57979  size_length_rugs_runners         n/a  bedbathandbeyond    69672237   
57980  size_length_rugs_runners         n/a  bedbathandbeyond    69672241   

                                                             long_desc  \
52                Capel Rugs® Gava Stripes 5\" x 8\" Area Rug in Beige   
68              Weather Guard™ Squares 34.5\" x 59\" Door Mat in Camel   
69         Weather Guard™ Squares 34.5\" x 59\" Door Mat in Dark Brown   
70           Weather Guard™ Squares 34.5\" x 59\" Door Mat in Charcoal   
71        Weather Guard™ Squares 34.5\" x 59\" Door Mat in Medium Grey   
...                                                                ...   
57976  The Farm Rooster 18\" x 30\" Anti-Fatigue Kitchen Mat in Walnut   
57977  Jolly Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Rain Drum   
57978  The Farm Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   
57979                    Wamsutta® Preston 24\" x 40\" Bath Rug in Sea   
57980                         Preston 17\" x 24\" Bath Rug in New Blue   

      resolution      matches  length  width  
52      standard    5\" x 8\"       5      8  
68      standard   5\" x 59\"       5     59  
69      standard   5\" x 59\"       5     59  
70      standard   5\" x 59\"       5     59  
71      standard   5\" x 59\"       5     59  
...          ...          ...     ...    ...  
57976   standard  18\" x 30\"      18     30  
57977   standard  22\" x 35\"      22     35  
57978   standard  22\" x 35\"      22     35  
57979   standard  24\" x 40\"      24     40  
57980   standard  17\" x 24\"      17     24  

[7343 rows x 9 columns]

In [14]:
x['Max'] = x[['length', 'width']].values.max(1)
x['Min'] = x[['length', 'width']].values.min(1)
x

<ipython-input-14-1d2e2fecdef1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Max'] = x[['length', 'width']].values.max(1)
<ipython-input-14-1d2e2fecdef1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Min'] = x[['length', 'width']].values.min(1)


attribute       value    customers_name external_id  \
52     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    61327016   
68     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    63730005   
69     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    63730043   
70     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    63730081   
71     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    63730142   
...                         ...         ...               ...         ...   
57976  size_length_rugs_runners         n/a  bedbathandbeyond    69649155   
57977  size_length_rugs_runners         n/a  bedbathandbeyond    69649156   
57978  size_length_rugs_runners         n/a  bedbathandbeyond    69649157   
57979  size_length_rugs_runners         n/a  bedbathandbeyond    69672237   
57980  size_length_rugs_runners         n/a  bedbathandbeyond    69672241   

                                                             long_desc  \
52                Capel Rugs® Gava Stripes 5\" x 8\" Area Rug in Beige   
68              Weather Guard™ Squares 34.5\" x 59\" Door Mat in Camel   
69         Weather Guard™ Squares 34.5\" x 59\" Door Mat in Dark Brown   
70           Weather Guard™ Squares 34.5\" x 59\" Door Mat in Charcoal   
71        Weather Guard™ Squares 34.5\" x 59\" Door Mat in Medium Grey   
...                                                                ...   
57976  The Farm Rooster 18\" x 30\" Anti-Fatigue Kitchen Mat in Walnut   
57977  Jolly Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Rain Drum   
57978  The Farm Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   
57979                    Wamsutta® Preston 24\" x 40\" Bath Rug in Sea   
57980                         Preston 17\" x 24\" Bath Rug in New Blue   

      resolution      matches  length  width  Max  Min  
52      standard    5\" x 8\"       5      8    8    5  
68      standard   5\" x 59\"       5     59   59    5  
69      standard   5\" x 59\"       5     59   59    5  
70      standard   5\" x 59\"       5     59   59    5  
71      standard   5\" x 59\"       5     59   59    5  
...          ...          ...     ...    ...  ...  ...  
57976   standard  18\" x 30\"      18     30   30   18  
57977   standard  22\" x 35\"      22     35   35   22  
57978   standard  22\" x 35\"      22     35   35   22  
57979   standard  24\" x 40\"      24     40   40   24  
57980   standard  17\" x 24\"      17     24   24   17  

[7343 rows x 11 columns]

In [15]:
# x['Max']=x['Max'].astype(str)
# x['Final']=x['Max'].apply(lambda x: kwp.extract_keywords(x))
# x

In [16]:
matches=x
matches

attribute       value    customers_name external_id  \
52     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    61327016   
68     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    63730005   
69     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    63730043   
70     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    63730081   
71     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond    63730142   
...                         ...         ...               ...         ...   
57976  size_length_rugs_runners         n/a  bedbathandbeyond    69649155   
57977  size_length_rugs_runners         n/a  bedbathandbeyond    69649156   
57978  size_length_rugs_runners         n/a  bedbathandbeyond    69649157   
57979  size_length_rugs_runners         n/a  bedbathandbeyond    69672237   
57980  size_length_rugs_runners         n/a  bedbathandbeyond    69672241   

                                                             long_desc  \
52                Capel Rugs® Gava Stripes 5\" x 8\" Area Rug in Beige   
68              Weather Guard™ Squares 34.5\" x 59\" Door Mat in Camel   
69         Weather Guard™ Squares 34.5\" x 59\" Door Mat in Dark Brown   
70           Weather Guard™ Squares 34.5\" x 59\" Door Mat in Charcoal   
71        Weather Guard™ Squares 34.5\" x 59\" Door Mat in Medium Grey   
...                                                                ...   
57976  The Farm Rooster 18\" x 30\" Anti-Fatigue Kitchen Mat in Walnut   
57977  Jolly Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Rain Drum   
57978  The Farm Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   
57979                    Wamsutta® Preston 24\" x 40\" Bath Rug in Sea   
57980                         Preston 17\" x 24\" Bath Rug in New Blue   

      resolution      matches  length  width  Max  Min  
52      standard    5\" x 8\"       5      8    8    5  
68      standard   5\" x 59\"       5     59   59    5  
69      standard   5\" x 59\"       5     59   59    5  
70      standard   5\" x 59\"       5     59   59    5  
71      standard   5\" x 59\"       5     59   59    5  
...          ...          ...     ...    ...  ...  ...  
57976   standard  18\" x 30\"      18     30   30   18  
57977   standard  22\" x 35\"      22     35   35   22  
57978   standard  22\" x 35\"      22     35   35   22  
57979   standard  24\" x 40\"      24     40   40   24  
57980   standard  17\" x 24\"      17     24   24   17  

[7343 rows x 11 columns]

In [17]:
# string = range_string
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

In [18]:
string = range_string
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return f'{lower}" - {upper}"{metric}'
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return f'{original_val}"{metric}'
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)


def range_app(num):

    if num >=  12 and num <= 23.9:
        return f' 12" - 23.9"'
        
    if num >=  24 and num <= 35.9:
        return f' 24" - 35.9"'
        
    if num >=  36 and num <= 47.9:
        return f' 36" - 47.9"'
        
    if num >=  48 and num <= 59.9:
        return f' 48" - 59.9"'
        
    if num >=  60 and num <= 71.9:
        return f' 60" - 71.9"'
        
    if num >=  72 and num <= 83.9:
        return f' 72" - 83.9"'
        
    if num >=  84 and num <= 95.9:
        return f' 84" - 95.9"'
        
    if num >=  96 and num <= 107.9:
        return f' 96" - 107.9"'
        
    if num >=  0 and num <= 11.9:
        return f' 0" - 11.9"'
        
    else:
        return r"n/a"



In [19]:
# matches['Min']=matches['Min'].astype(int)
# matches['final']=matches['Max'].apply(lambda x: kwp.extract_keywords(x))
# matches.dtypes

In [20]:
matches[curation_col]=matches['Max'].apply(range_app)
matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

<ipython-input-20-83791ecbab1c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches[curation_col]=matches['Max'].apply(range_app)
<ipython-input-20-83791ecbab1c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['external_id']=matches['external_id'].apply('="{}"'.format)


attribute       value    customers_name  external_id  \
52     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond  ="61327016"   
68     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond  ="63730005"   
69     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond  ="63730043"   
70     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond  ="63730081"   
71     size_length_rugs_runners  0" - 11.9"  bedbathandbeyond  ="63730142"   
...                         ...         ...               ...          ...   
57976  size_length_rugs_runners         n/a  bedbathandbeyond  ="69649155"   
57977  size_length_rugs_runners         n/a  bedbathandbeyond  ="69649156"   
57978  size_length_rugs_runners         n/a  bedbathandbeyond  ="69649157"   
57979  size_length_rugs_runners         n/a  bedbathandbeyond  ="69672237"   
57980  size_length_rugs_runners         n/a  bedbathandbeyond  ="69672241"   

                                                             long_desc  \
52                Capel Rugs® Gava Stripes 5\" x 8\" Area Rug in Beige   
68              Weather Guard™ Squares 34.5\" x 59\" Door Mat in Camel   
69         Weather Guard™ Squares 34.5\" x 59\" Door Mat in Dark Brown   
70           Weather Guard™ Squares 34.5\" x 59\" Door Mat in Charcoal   
71        Weather Guard™ Squares 34.5\" x 59\" Door Mat in Medium Grey   
...                                                                ...   
57976  The Farm Rooster 18\" x 30\" Anti-Fatigue Kitchen Mat in Walnut   
57977  Jolly Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Rain Drum   
57978  The Farm Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   
57979                    Wamsutta® Preston 24\" x 40\" Bath Rug in Sea   
57980                         Preston 17\" x 24\" Bath Rug in New Blue   

      resolution      matches  length  width  Max  Min  \
52      standard    5\" x 8\"       5      8    8    5   
68      standard   5\" x 59\"       5     59   59    5   
69      standard   5\" x 59\"       5     59   59    5   
70      standard   5\" x 59\"       5     59   59    5   
71      standard   5\" x 59\"       5     59   59    5   
...          ...          ...     ...    ...  ...  ...   
57976   standard  18\" x 30\"      18     30   30   18   
57977   standard  22\" x 35\"      22     35   35   22   
57978   standard  22\" x 35\"      22     35   35   22   
57979   standard  24\" x 40\"      24     40   40   24   
57980   standard  17\" x 24\"      17     24   24   17   

      Q:size_length_rugs_runners  
52                    0" - 11.9"  
68                   48" - 59.9"  
69                   48" - 59.9"  
70                   48" - 59.9"  
71                   48" - 59.9"  
...                          ...  
57976                24" - 35.9"  
57977                24" - 35.9"  
57978                24" - 35.9"  
57979                36" - 47.9"  
57980                24" - 35.9"  

[7343 rows x 12 columns]

In [21]:
error----Switch Max and Min when doing length and width

SyntaxError: invalid syntax (<ipython-input-21-7509e0f4fe46>, line 1)

In [22]:
gc = gcloud.Gsheets()

In [23]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

succesfully saved to worksheet:`ul` 
 url = `https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0`


# send to the folder for upload

In [24]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na {buckets} Shawn.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match {buckets} Shawn.csv',index=False)

In [25]:
looks_good('Bed Bath & Beyond', attribute, df, matches)

<ipython-input-24-f8dda3983948>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_matches[curation_col] = r'n/a'
